## Stream de Dados do Twitter com MongoDB, Pandas e Scikit Learn

In [ ]:
'''1. Acessar o site: https://apps.twitter.com/
2. Solicitar uma conta de desenvolvedor do twitter. O twitter vai perguntar em que você está interessado em 
desenvolver e pedir pra que você descreva sua aplicação em 300 caracteres.
3. No check box 'Utiliza dados do governo?' selecione 'No'.
4. Leia e concorde os termos de serviço e confirme sua inscrição. 
5. Confirme seu endereço de e-mail e aguarde a aprovação (pode levar algumas semanas).
6. Estando aprovado, acesse sua conta na area appps twitter e comece a criar sua aplicação. As keys criadas
serão inseridas nos comandos abaixo.'''

In [ ]:
# Instala o pacote tweepy
!pip install tweepy

In [ ]:
# Importando os módulos Tweepy, Datetime e Json
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
import json

In [ ]:
# Adicione aqui sua Consumer Key
consumer_key = "xxxxxxxxx"

In [ ]:
# Adicione aqui sua Consumer Secret 
consumer_secret = "xxxxxxxxx"

In [ ]:
# Adicione aqui seu Access Token
access_token = "xxxxxxxxx"

In [ ]:
# Adicione aqui seu Access Token Secret
access_token_secret = "xxxxxxxxx"

In [ ]:
# Criando as chaves de autenticação
auth = OAuthHandler(consumer_key, consumer_secret)

In [ ]:
auth.set_access_token(access_token, access_token_secret)

In [ ]:
# Criando uma classe para capturar os stream de dados do Twitter e 
# armazenar no MongoDB
class MyListener(StreamListener):
    def on_data(self, dados):
        tweet = json.loads(dados)
        created_at = tweet["created_at"]
        id_str = tweet["id_str"]
        text = tweet["text"]
        obj = {"created_at":created_at,"id_str":id_str,"text":text,}
        tweetind = col.insert_one(obj).inserted_id
        print (obj)
        return True

In [ ]:
# Criando o objeto mylistener
mylistener = MyListener()

In [ ]:
# Criando o objeto mystream
mystream = Stream(auth, listener = mylistener)

In [ ]:
# Importando do PyMongo o módulo MongoClient
from pymongo import MongoClient   

In [ ]:
# Criando a conexão ao MongoDB
client = MongoClient('localhost', 27017)

In [ ]:
# Criando o banco de dados twitterdb
db = client.twitterdb

In [ ]:
# Criando a collection "col"
col = db.tweets 

In [ ]:
# Criando uma lista de palavras chave para buscar nos Tweets
keywords = ['Big Data', 'Python', 'Data Mining', 'Data Science']

## Coletando os Tweets

In [ ]:
# Iniciando o filtro e gravando os tweets no MongoDB
mystream.filter(track=keywords)

In [ ]:
mystream.disconnect()

In [ ]:
# Verificando um documento no collection
col.find_one()

## Análise de Dados com Pandas e Scikit-Learn

In [ ]:
# criando um dataset com dados retornados do MongoDB
dataset = [{"created_at": item["created_at"], "text": item["text"],} for item in col.find()]

In [ ]:
# Importando o módulo Pandas 
import pandas as pd
pd.__version__

In [ ]:
# Criando um dataframe a partir do dataset 
df = pd.DataFrame(dataset)

In [ ]:
# Imprimindo o dataframe
df

In [ ]:
# Importando o módulo Scikit Learn
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
# Usando o método CountVectorizer para criar uma matriz de documentos
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.text)

In [ ]:
# Contando o número de ocorrências das principais palavras do dataset
word_count = pd.DataFrame(cv.get_feature_names(), columns=["word"])
word_count["count"] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)
word_count[:50]